In [1]:
import numpy as np
import pandas as pd
import collections

import re

In [2]:
f = open('inputs/day07.txt')
data = f.read()

In [3]:
def flatten(l):
    for el in l:
        if isinstance(el, collections.Iterable) and not isinstance(el, (str, bytes)):
            yield from flatten(el)
        else:
            yield el

class InstructionTree:
    
    def __init__(self, instructions):
        self.data = data
        self.node_names = list(pd.Series(list(flatten(re.findall('(?<= )[A-Z](?= )', data)))).sort_values().unique())
        self.nodes = {}
        for n in self.node_names:
            self.nodes[n] = Node(n)
        self.instructions = data.split('\n')
        self.completed = False
        self.completed_nodes = []
        self.next_available_nodes = []
    
    def parse_line(self, line):
        names = re.findall('(?<= )[A-Z](?= )', line)    
        return names[0], names[1]
    
    def build_tree(self):
        for line in self.instructions:
            n0, n1 = self.parse_line(line)
            self.nodes[n0].add_child(self.nodes[n1])
            self.nodes[n1].add_parent(self.nodes[n0])
        for nm in self.node_names:
            node = self.nodes[nm]
            if not node.has_parent():
                self.root = node
                self.next_available_nodes.append(node)
        self.next_available_nodes.sort()
        self.current = self.next_available_nodes.pop(0)
        
    def completed_parents(self, node):
        return all([parent in self.completed_nodes for parent in node.parents])
            
    def find_next_available_nodes(self, node):
        next_list = []
        # Determine next available nodes
        # Check that the current node has children to add
        if node.has_children():
            # only add children if all of their parents have been completed
            for child in node.children:
                if self.completed_parents(child):
                    next_list.append(child)
        # resort the list
        return next_list
        
    def move_to_next_node(self):
        # Complete the current node
        self.completed_nodes.append(self.current)

        # Find next available nodes
        self.next_available_nodes.append(self.find_next_available_nodes(self.current))
        self.next_available_nodes = list(flatten(self.next_available_nodes))
        self.next_available_nodes.sort()    
        print(self.current, self.next_available_nodes)
        # Change the current node to the next node
        if len(self.next_available_nodes) > 0:
            self.current = self.next_available_nodes.pop(0)
        else:
            self.completed = True
        
class Node:

    def __init__(self, name):
        self.name = name
        self.parents = []
        self.children = []
    
    def __repr__(self):
        return self.name
    
    def __gt__(self, other):
        self.name > other.name
        
    def __eq__(self, other):
        return self.name == other.name

    def __lt__(self, other):
        return self.name < other.name
    
    def add_parent(self, parent):
        self.parents.append(parent)
        
    def has_parent(self):
        return len(self.parents) > 0
        
    def add_child(self, child):
        self.children.append(child)
        
    def has_children(self):
        return len(self.children) > 0

In [4]:
tree = InstructionTree(data)
tree.build_tree()

In [5]:
while not tree.completed:
    tree.move_to_next_node()
''.join([n.name for n in tree.completed_nodes])

G [D, O, S, U, X]
D [H, O, S, U, X]
H [O, S, U, X]
O [S, U, X]
S [U, X]
U [X]
X [A]
A [C, I, M, R]
C [I, M, R, T]
I [M, R, T, W]
M [R, T, W]
R [T, W]
T [P, W]
P [W]
W [N, Y]
N [Y]
Y [J]
J [L]
L [E, Q]
E [Q, V]
Q [F, V]
F [V]
V [Z]
Z [B]
B [K]
K []


'GDHOSUXACIMRTPWNYJLEQFVZBK'